# Imports

In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 \
!pip3 install lightning pillow numpy opencv-python ultralytics cython
!pip3 install pytorchyolo
!pip3 install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"

ERROR: Invalid requirement: '!pip3'


  Cloning https://github.com/philferriere/cocoapi.git to c:\users\timhi\appdata\local\temp\pip-install-x9ctqldt\pycocotools_5220e0c7ddf7402ca85dee7b257ae94f
  Resolved https://github.com/philferriere/cocoapi.git to commit 2929bd2ef6b451054755dfd7ceb09278f935f7ad
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/philferriere/cocoapi.git 'C:\Users\timhi\AppData\Local\Temp\pip-install-x9ctqldt\pycocotools_5220e0c7ddf7402ca85dee7b257ae94f'


In [2]:
import os
import json
import shutil
import random
from pathlib import Path

import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools.coco import COCO
from torch.utils.data import Dataset

coco_path = "tissue-cells/tiger-coco.json" #"tissue-cells/tiger-coco.json"#"tissue-cells/tiger-coco.json"
tissue_cell_path = "tissue-cells/images"#"tissue-cells/images"#"tissue-cells/images"
coco = COCO(coco_path)

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!


# Auxillary functions

In [3]:
def draw_bbox(image, bbox_list, relative=False):
  fig, ax = plt.subplots()
  h, w, _ = image.shape
  
  # Draw boxes and add label to each box
  for box in bbox_list:
    if relative:
      abs_box = [int((box[0]-box[2]/2)*w), int((box[1]-box[3]/2)*h), int(box[2]*w), int(box[3]*h)]
      bb = patches.Rectangle((abs_box[0],abs_box[1]), abs_box[2],abs_box[3], linewidth=2, edgecolor="blue", facecolor="none")
    else:
      bb = patches.Rectangle((box[0],box[1]), box[2],box[3], linewidth=2, edgecolor="blue", facecolor="none")
    ax.add_patch(bb)

  ax.imshow(image)
  plt.show()


def draw_file(data, file_name):
  images_path = "data/roi-level-annotations/tissue-cells/images/"#"tissue-cells/images/"
  image_name = file_name
  image = Image.open(images_path+image_name)

  parsed_file_name = './images/' + file_name
  
  for i in data['images']:
    if i['file_name'] == parsed_file_name:
      img_id = i['id']
  
  anns = coco.loadAnns(coco.getAnnIds(imgIds=img_id))

  draw_bbox(image, [ann['bbox'] for ann in anns])
  

def get_bboxes(data, file_name):
  parsed_file_name = './images/' + file_name
  
  for i in data['images']:
    if i['file_name'] == parsed_file_name:
      img_id = i['id']
  
  anns = coco.loadAnns(coco.getAnnIds(imgIds=img_id))

  result = []
  
  # Draw boxes and add label to each box
  for ann in anns:
      box = ann['bbox']
      result.append(box)
  
  return result

with open(coco_path, 'r', encoding = 'utf-8') as f:
  data = json.load(f)

# Loading in images in a dataset structure

In [4]:
class Tiger(Dataset):
    def __init__(self, image_dir, transform=None):
        self.images = self.load_images(image_dir)
        self.case_ids = list(self.images.keys())
        self.bbox_list = self.load_bbox_list()
        self.transform = transform
        
    def load_images(self, data_dir):
        images = {}
        for image_path in data_dir.iterdir():
            case_id = image_path.stem
            images[case_id] = image_path
            
        return images
    
    def load_image(self, case_id):
        with Image.open(self.images[case_id]) as im:
            image = np.array(im).astype(np.uint8)
        return image
    
    def load_bbox_list(self):
        bbox_list = {}
        for case_id in self.case_ids:
            bbox_list[case_id] = self.load_case_bbox_list(case_id)
        return bbox_list
    
    def load_case_bbox_list(self, case_id):
        bbox_list = get_bboxes(data,case_id+'.png')
        return bbox_list
    
    def visualize(self, idx):
        image, bbox_list = self[idx]
        
        # Remove channel dimension
        image = image[0]
        
        # Plot x-ray
        fig, ax = plt.subplots()
        ax.imshow(image, cmap="gray")
        
        # Plot box
        for bbox in bbox_list:
            bb = patches.Rectangle((bbox[0],bbox[1]), bbox[2],bbox[3], linewidth=2, edgecolor="blue", facecolor="none")
            ax.add_patch(bb)
        
        plt.show()

    def visualize_color(self, idx):
        image, bbox_list = self[idx]
        draw_bbox(image.transpose(1, 2, 0), bbox_list)
    
    def __getitem__(self, idx):
        case_id = self.case_ids[idx]
        image = self.load_image(case_id) / 255  
        bbox_list = self.bbox_list[case_id]
        
        # Add channel dimension to image
        image = np.transpose(image, (2,0,1))
        
        if self.transform:
            image = self.transform(image)

        return image, bbox_list

    def __len__(self):
        return len(self.images)


In [5]:
tiger_dataset = Tiger(Path(tissue_cell_path))
print(len(tiger_dataset), "images loaded")

1879 images loaded


# Create a dataset in the style of yolo7

In [6]:
def rotate90(relative_boxes, image_height, image_width):
    rotated90_boxes = []
    for box in relative_boxes:
        relative_x, relative_y, relative_w, relative_h = box
        
        rotated90_x = 1 - relative_y
        rotated90_y = relative_x
        rotated90_w = relative_h
        rotated90_h = relative_w
        
        rotated90_boxes.append((rotated90_x, rotated90_y, rotated90_w, rotated90_h))
        
    return rotated90_boxes

In [7]:
def rotate180(relative_boxes, image_height, image_width):
    rotated180_boxes = []
    for box in relative_boxes:
        relative_x, relative_y, relative_w, relative_h = box
        
        rotated180_x = 1 - relative_x
        rotated180_y = 1 - relative_y
        
        rotated180_boxes.append((rotated180_x, rotated180_y, relative_w, relative_h))
        
    return rotated180_boxes

In [8]:
def rotate270(relative_boxes, image_height, image_width):
    rotated270_boxes = []
    for box in relative_boxes:
        relative_x, relative_y, relative_w, relative_h = box
        
        rotated270_x = relative_y
        rotated270_y = 1 - relative_x
        rotated270_w = relative_h
        rotated270_h = relative_w
        
        rotated270_boxes.append((rotated270_x, rotated270_y, rotated270_w, rotated270_h))
        
    return rotated270_boxes

In [9]:
def save_image_and_bbox(img, path, path_addition, bbox_list):
    img.save(path+path_addition+".jpg")
        
    with open(path+path_addition+".txt", 'w') as file:
        for bbox in bbox_list:
            file.write("0 "+' '.join([str(d) for d in bbox])+'\n')

In [10]:
def convert_to_relative(bounding_boxes, img_height, img_width):
    relative_boxes = []
    for box in bounding_boxes:
        x, y, w, h = box
        if x < -w/5 or y < -h/5 or x > (img_width - w*0.8) or y > (img_height - h*0.8):
            continue

        # first limit the bbox to the image dims
        new_x = max(x, 0)
        new_y = max(y, 0)
        
        new_w = min(w+(x-new_x), img_width-new_x-1)
        new_h = min(h+(y-new_y), img_height-new_y-1)

        # original code
        rel_x = (new_x + (new_w/2)+1)/ img_width
        rel_y = (new_y + (new_h/2)+1)/ img_height
        rel_w = new_w / img_width
        rel_h = new_h / img_height

        rel_box = (rel_x, rel_y, rel_w, rel_h)

        relative_boxes.append(rel_box)
    return relative_boxes

os.makedirs('temp', exist_ok=True)
count = 0

for i in tqdm(range(len(tiger_dataset))):#range(30))
    image_array, bbox_list = tiger_dataset[i]
    bbox_list = convert_to_relative(bbox_list, *image_array.shape[1:])

    path = f"temp/{i}"
    image_array_trans = (image_array*255).astype(np.uint8).transpose(1, 2, 0)
    image = Image.fromarray(image_array_trans)
    
    save_image_and_bbox(image, path, '', bbox_list)
    
    # draw_bbox(image_array, bbox_list, relative=True)
    # tiger_dataset.visualize_color(i)
                
    random_float = random.random()   
    thres = 0.5
    fraction = thres/8
    if random_float < thres:       
        #Rotate images
        #Rotate 90 degrees
        if random_float < fraction: # 1/8
            rot90_bbox_list = rotate90(bbox_list, *image_array.shape[1:])
            rot90_img = image.transpose(Image.ROTATE_90)
            save_image_and_bbox(rot90_img, path, 'r90', rot90_bbox_list)
        
        #Rotate 180 degrees
        elif random_float < fraction*2: # 2/8
            rot180_bbox_list = rotate180(bbox_list, *image_array.shape[1:])
            rot180_img = image.transpose(Image.ROTATE_180)
            save_image_and_bbox(rot180_img, path, 'r180', rot180_bbox_list)
        
        #Rotate 270 degrees
        elif random_float < fraction*3: # 3/8
            rot270_bbox_list = rotate270(bbox_list, *image_array.shape[1:])
            rot270_img = image.transpose(Image.ROTATE_270)
            save_image_and_bbox(rot270_img, path, 'r270', rot270_bbox_list)
        
        #Data enhancement augmentations
        #Adjust sharpness
        elif random_float < fraction*4: # 4/8
            enhancer = ImageEnhance.Sharpness(image)
            sharp_img = enhancer.enhance(10.0)
            save_image_and_bbox(sharp_img, path, 'sharp', bbox_list)
        
        #Adjust contrast
        elif random_float < fraction*5: # 5/8
            enhancer = ImageEnhance.Contrast(image)
            contrast_img = enhancer.enhance(1.5)
            save_image_and_bbox(contrast_img, path, 'contrast', bbox_list)
                    
        #Add blur filter
        elif random_float < fraction*6: # 6/8
            blur_img = image.filter(ImageFilter.BLUR)
            save_image_and_bbox(blur_img, path, 'blur', bbox_list)
                    
        #Enhance edges
        elif random_float < fraction*7: # 7/8
            edge_img = image.filter(ImageFilter.EDGE_ENHANCE)
            save_image_and_bbox(edge_img, path, 'edge', bbox_list)
        
        #Add smoothing filter
        elif random_float < fraction*8: # 8/8
            smooth_img = image.filter(ImageFilter.SMOOTH_MORE)
            save_image_and_bbox(smooth_img, path, 'smooth', bbox_list)

100%|██████████| 1879/1879 [00:23<00:00, 80.92it/s] 


In [11]:
def split_directory(input_dir, output_dir, ratio):
    # Create train and test directories
    output_image_dir = output_dir+"/images"
    output_label_dir = output_dir+"/labels"
    locations = ['train', 'test', 'val']

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    for loc in locations:
        os.makedirs(output_image_dir+'/'+loc, exist_ok=True)
        os.makedirs(output_label_dir+'/'+loc, exist_ok=True)


    # Get a list of all files in the input directory
    files = os.listdir(input_dir)
    name_dict = {}

    for file in files:
        name, ext = os.path.splitext(file)
        name_dict.setdefault(name, []).append(ext)

    files_with_same_name = [name for name, exts in name_dict.items() if len(exts) > 1]

    # Shuffle the files randomly
    random.shuffle(files_with_same_name)

    # Calculate the number of files for training and testing
    train_count = int(len(files_with_same_name) * ratio)
    test_count = train_count+(len(files_with_same_name) - train_count)//2

    
    train_files = files_with_same_name[:train_count]
    test_files = files_with_same_name[train_count:test_count]
    val_files = files_with_same_name[test_count:]
    print("train:",len(train_files), 
          "test:",len(test_files),
          "val:",len(val_files))

    # Move files to the respective train and test directories
    def move_files(files, loc):
        for file in files:
            shutil.move(
                os.path.join(input_dir, file+'.jpg'),
                os.path.join(output_image_dir+'/'+loc, file+'.jpg'))
            shutil.move(
                os.path.join(input_dir, file+'.txt'),
                os.path.join(output_label_dir+'/'+loc, file+'.txt'))

    move_files(train_files, locations[0])
    move_files(test_files, locations[1])
    move_files(val_files, locations[2])
    os.rmdir(input_dir)

# Example usage
input_directory = 'temp'
output_directory = 'cell_dataset'
train_ratio = 0.8  # 80% train, 10% test, 10% validation

split_directory(input_directory, output_directory, train_ratio)

train: 2241 test: 280 val: 281
